In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPool2D, BatchNormalization 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models, losses
import numpy as np 

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 6s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
x_train = x_train.astype('float32') / 255.0 
x_test = x_test.astype('float32') / 255.0 

In [ ]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
x_train.shape[1:]

(32, 32, 3)

In [ ]:
model = Sequential()
model.add(layers.experimental.preprocessing.Resizing(227, 227, interpolation="bilinear", input_shape=x_train.shape[1:]))
model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='same'))
model.add(MaxPool2D((3,3), strides=2))
model.add(BatchNormalization())

model.add(Conv2D(256, (5,5), strides=1, activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), strides=2))
model.add(BatchNormalization())

model.add(Conv2D(384, (3,3), strides=1, activation='relu', padding='same'))
model.add(Conv2D(384, (3,3), strides=1, activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), strides=1, activation='relu', padding='same'))
model.add(MaxPool2D((3,3), strides=2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 57, 57, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 256)       1

In [ ]:
model.compile(optimizer='adam', 
              loss=losses.categorical_crossentropy, 
              metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

Epoch 1/50
391/391 [==============================] - 155s 307ms/step - loss: 1.8050 - accuracy: 0.3926 - val_loss: 2.8858 - val_accuracy: 0.2663
Epoch 2/50
391/391 [==============================] - 115s 294ms/step - loss: 1.2629 - accuracy: 0.5533 - val_loss: 1.4235 - val_accuracy: 0.5093
Epoch 3/50
391/391 [==============================] - 115s 294ms/step - loss: 1.0539 - accuracy: 0.6348 - val_loss: 1.0069 - val_accuracy: 0.6511
Epoch 4/50
391/391 [==============================] - 115s 294ms/step - loss: 0.9133 - accuracy: 0.6850 - val_loss: 1.6661 - val_accuracy: 0.4328
Epoch 5/50
391/391 [==============================] - 115s 294ms/step - loss: 0.8144 - accuracy: 0.7214 - val_loss: 0.8697 - val_accuracy: 0.7041
Epoch 6/50
391/391 [==============================] - 115s 295ms/step - loss: 0.6999 - accuracy: 0.7604 - val_loss: 1.0037 - val_accuracy: 0.6494
Epoch 7/50
391/391 [==============================] - 115s 295ms/step - loss: 0.6536 - accuracy: 0.7761 - val_loss: 0.7447 -

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['epoch', 'validation'], loc = 'upper left')
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
predictions = model.predict(x_test, verbose=1)

In [ ]:
def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def display_image_predictions(features, labels, predictions):
    n_classes = 10
    label_names = load_label_names()
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(range(n_classes))
    label_ids = label_binarizer.inverse_transform(np.array(labels))

    fig, axs = plt.subplots(10, 2, figsize=(12,24))
    margin = 0.05
    ind = np.arange(n_classes)
    width = (1. - 2. * margin) / n_classes    
    arr_features = [None]*10
    arr_predictions = np.zeros((10, 10))
    last_image_i = 0
    
    for i, prediction in enumerate(predictions):
        label_id = label_ids[i]
        feature = features[i]
        arr_features[label_id] = feature
        max_id = np.argmax(prediction)
        arr_predictions[label_id][max_id] += 1
        last_image_i = i

    arr_predictions /= last_image_i
    
    for i in range(10):
      feature = arr_features[i]
      pred_name = label_names[i]
      prediction = arr_predictions[i]
      axs[i][0].imshow(feature)
      axs[i][0].set_title(pred_name)
      axs[i][0].set_axis_off()

      axs[i][1].barh(ind + margin, prediction, width)
      axs[i][1].set_yticks(ind + margin)
      axs[i][1].set_yticklabels(label_names)

    plt.tight_layout()